# 获取二手房信息代码：

In [130]:
import tkinter as tk
from tkinter import ttk
import tkinter.messagebox
import pinyin
import pandas as pd
import time
import logging
import difflib
import random
import sys
from ttkbootstrap import Style
import csv
import getopt
import argparse
import requests
import sqlite3
import parsel
from lxml import etree
from tkinter import filedialog

def fuzzy_matching(texts, value):
    texts = texts.split(",")
    texts_score = {}
    logging.info("展开模糊搜索成功！\n")
    for i in texts:
        score = difflib.SequenceMatcher(None, i, value).quick_ratio()
        texts_score[i] = score
    texts_score = sorted(texts_score.items(), key=lambda x: x[1], reverse=False)
    logging.info("获取搜索匹配数据成功，且对数据进行排序！\n")
    po=1
    tmp12=0
    if flagofsearch==1 :
        b=[x[1] for x in texts_score]
        for i in range(100):
            if(b[-i]>0):
                print(i,end=':')
                print(texts_score[-i])
                tmp12=1
        logging.info("成功输出获取数据供给用户选择！(开启了模糊匹配选项)\n")
        if(tmp12==1):
            po=int(input("选择你所想输入的城市名称（1-N）:"))
            match_value = texts_score[-po][0]
            return match_value
        else:
            return -1    
    else :
        # print('\n*****************',texts_score[-1][1],'*****************\n')
        if texts_score[-1][1]==1 :
            return texts_score[-1][0]
        else :
            return -1

def get_pinyin_first_alpha(name):
    return "".join([i[0] for i in pinyin.get(name, " ").split(" ")])


class House:
    def __init__(self,master):
        self.columns={
        'ID':25,
        '城市':40,
        '广告标题':350,
        '地址':200,
        '总价格':50,
        '每平方的价格':100,
        '主要信息':450
        }
        self.tree=ttk.Treeview(master,height=18,show='headings')
        self.tree['columns'] = list(self.columns)
        for column in self.columns:  # 批量设置列属性
            self.tree.heading(column, text=column)  # #设置列标题
            self.tree.column(column, width=self.columns[column], anchor='center')  
        
        self.maxpage=50 #最大搜索页数 一页一般有三十个数据 这里默认最多50页 gui无法更改
        self.maxnumbers=30 #最大搜索数据条数 一般设置30
        
        self.savecheckbt=tk.Checkbutton(master,text='存储数据到excle文件',command=self.funchooseexcle)
        self.savecheckbt2=tk.Checkbutton(master,text='存储数据到数据库',command=self.funchoosedatab)
        self.savecheckbt.select()
        self.savemethod=1  #保存方式 有excle和database两种

        self.savecheckbt.grid(row=3,column=4)
        self.savecheckbt2.grid(row=3,column=4,sticky='e')
        self.progressbar=ttk.Progressbar(master,length=600)
        self.progressbar.grid(row=3,column=1,columnspan=5,pady=10,sticky='w')
        self.l2=tk.Label(master,text='搜索进度:',font=('微软雅黑',12))
        self.l2.grid(row=3,column=0)

        self.l3=tk.Label(master,text='加载进度:',font=('微软雅黑',12))
        self.l3.grid(row=2,column=0)

        self.progressbar2=ttk.Progressbar(master,length=200)
        self.progressbar2.grid(row=2,column=1,columnspan=2,pady=1)
        self.progressbar2['maximum']=self.maxnumbers

        self.progressbar['maximum']=100
        # self.canvas=tk.Canvas(master,width=600,height=15,bg='black')
        # self.canvas.grid(row=3,column=0,columnspan=3,pady=10)
        self.titles = []
        self.addresses = []
        self.introduces = []
        self.totalprices = []
        self.unitprices = []
        self.ress = []

        self.ds=[]

        self.tree.grid(row=1,column=0,columnspan=6) 
       
        
        self.b1 = tk.Label(master,text='请输入省会城市的名称:',font=('微软雅黑',12))
        self.b1.grid(row=0,column=3)

        self.e =tk.Entry(master,show=None,width=100)
        self.e.grid(row=0,column=4) 

        v=tk.StringVar(value='30')
        self.e2 =tk.Entry(master,show=None,width=10,textvariable=v)
        self.e2.grid(row=0,column=1,sticky='w')

        self.bt3=tk.Button(text='设置最大搜索数量:',command=self.setmaxnumbers)
        self.bt3.grid(row=0,column=0,sticky='e')


        # self.bt1=tk.Button(master,text='搜索',width=8,height=1,command=lambda:[self.progress(),self.search()])
        self.bt1=tk.Button(master,text='查询',width=8,height=1,command=lambda:[self.showprogress(),self.search()])

        self.bt1.grid(row=0,column=5)  
        self.btnDelete1=tkinter.Button(master,text='删除所选数据',font=('微软雅黑',12),command=self.deleteclick,width=10,height=1)
        self.btnDelete1.grid(row=2,column=3)  #
        self.btnDelete2=tkinter.Button(master,text='删除所有数据',font=('微软雅黑',12),command=self.deletealldata,width=10,height=1)
        self.btnDelete2.grid(row=2,column=4)
        self.button4 = tk.Button(master,text='保存所有数据',font=('微软雅黑',12),command=self.save,width=10,height=1)
        self.button4.grid(row=2,column=5)
    def setmaxnumbers(self):
        self.maxnumbers=int(self.e2.get())
        tkinter.messagebox.showinfo('提示','最大搜索数据数量设置成功')
        print('ok')
    
    def funchooseexcle(self):
        self.savemethod=1
        self.savecheckbt.select()
        self.savecheckbt2.deselect()
    def funchoosedatab(self):
        self.savemethod=0
        self.savecheckbt.deselect()
        self.savecheckbt2.select()


    def deleteclick(self):
        if not self.tree.selection():
            tkinter.messagebox.showerror('抱歉','你还没有选择，无法删除数据')
            return
        for item in self.tree.selection():
            self.tree.delete(item)

    def showprogress(self):
        for i in range(100):
            self.progressbar['value']=i+1
            window.update()
            time.sleep(0.005)
    

    def deletealldata(self):
        global cnt
        def delButton(tree):
            x=tree.get_children()
            for item in x:
                # print(item)
                tree.delete(item)
            logging.info('删除所有数据成功')
        delButton(self.tree)
        self.titles = []
        self.addresses = []
        self.introduces = []
        self.totalprices = []
        self.unitprices = []
        self.ress = []
        self.ds=[[]]
        cnt=0
    

    def search(self):
        global city
        global cnt
        self.progressbar2['maximum']=self.maxnumbers
        self.progressbar['value']=0
        flag1=0
        city =self.e.get()
        if city:

            
            res=fuzzy_matching(data1,city)
            if res!=-1:
                for page in range(1,100):
                    global datas
                    logging.info("模糊匹配完成！")
                    resultcity=get_pinyin_first_alpha(res)
                    logging.info("汉字拼音首字母获取成功！")
                    url='https://'+resultcity+f'.lianjia.com/ershoufang/pg{page}/'
                    headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36 Edg/107.0.1418.42'}
                    reponse = requests.get(url=url,headers=headers)
                    html_data = reponse.text
                    #print(html_data)
                    selector = parsel.Selector(html_data)
                    lis = selector.css('.clear.LOGCLICKDATA')
                    #print(lis)
                    for li in lis:
                        if cnt > self.maxnumbers:
                            if flag1==0:
                                tkinter.messagebox.showinfo("提示",'已达到最大搜索数据上限，请在左上角进行添加以便搜索更多数据\n设置完成记得点击按钮确认')

                            datas.columns=['city','title','address','price','price_pre_squ','houseInfo']
                            logging.info('爬取信息成功')
                            return datas

                        title = li.css('.title a::text').get()
                        
                        self.progressbar2['value']=cnt
                        id=str(cnt)
                        address = '-'.join(li.css('.positionInfo a::text').getall())
                        introduce = li.css('.houseInfo::text').get()
                        totalprice = li.css('.priceInfo .totalPrice span::text').get()+'万'
                        unitprict = li.css('.unitPrice span::text').get()
                        Data=[id,res,title,address,totalprice,unitprict,introduce]
                        self.titles.append(title)
                        self.addresses.append(address)
                        self.introduces.append(introduce)
                        self.totalprices.append(totalprice)
                        self.unitprices.append(unitprict)
                        self.ress.append(res)
                        
                        self.ds.append([])
                        self.ds[cnt].append(res)
                        self.ds[cnt].append(title)
                        self.ds[cnt].append(address)
                        self.ds[cnt].append(totalprice)
                        self.ds[cnt].append(unitprict)
                        self.ds[cnt].append(introduce)
                        flag1=1
                        cnt+=1
                        
                        datas=pd.DataFrame([self.ress,self.titles,self.addresses,self.totalprices,self.unitprices,self.introduces]).T
                        self.tree.insert('', cnt,text='', values=Data)
                        if cnt%random.randrange(1,10)==0:
                            window.update()
                # datas.columns=['city','title','address','price','price_pre_squ','houseInfo']
                # logging.info('爬取信息成功')
                # return datas 
            else:
                tkinter.messagebox.showerror('输入错误',"没有找到相应的城市！可重新启动程序在初始化菜单选择4进行补充！")
        else :
            tkinter.messagebox.showinfo('错误信息','请输入内容')
    def save(self):
        self.df = pd.DataFrame([self.ress,self.titles,self.addresses,self.totalprices,self.unitprices,self.introduces]).T
        try:
            if self.savemethod==0:
                saveDataDB(self.ds,"house.db")
                tkinter.messagebox.showinfo('提示','保存数据成功\n可在py文件当前目录进行查看')
                # print([self.ress,self.titles,self.addresses,self.totalprices,self.unitprices,self.introduces])
                # output("house.db")
            else :
                savefile = filedialog.asksaveasfilename(filetypes=(("Excel files", "*.xlsx"), ("All files", "*.*") ))
                self.df.to_excel(savefile + ".xlsx", index=False, sheet_name="Results")
                logging.info('保存数据成功')
        except Exception as e:
            print(self.ds)
            tkinter.messagebox.showerror(title = '错误信息',message = str(e))


def init_db(dbpath):
    initsql = "drop table if exists houseinfotable" 

    createsql = '''
        create table if not exists houseinfotable
        (
            id integer primary key autoincrement ,
            city varchar ,
            title varchar ,
            address varchar ,
            price varchar ,
            price_pre_squ varchar,
            houseInfo varchar
        )
    '''                          

    conn = sqlite3.connect(dbpath)   #打开或创建 连接数据库文件
    cursor = conn.cursor()           #获取游标
    cursor.execute(initsql)          # 执行SQL语句
    cursor.execute(createsql)        #执行SQL语句
    conn.commit()                    #提交数据库操作
    conn.close()

def saveDataDB(datalist,dbpath):
    init_db(dbpath)                  #初始化数据库
    conn = sqlite3.connect(dbpath)   #连接数据库文件
    cur = conn.cursor()              #获取游标

    for data in datalist:
        cur.execute("insert into houseinfotable(city,title,address,price,price_pre_squ,houseInfo)values(?, ?, ?, ?, ?, ?)",( data[0], data[1], data[2], data[3] ,data[4], data[5]))     #执行SQL语句

    conn.commit()      #提交数据库操作
    cur.close()
    conn.close()       #关闭数据库连接

def output(dbpath):
    con = sqlite3.connect(dbpath)
    cur = con.cursor()
    sql = "select * from studentTable"
    datas = cur.execute(sql)
    for data in datas:
        print(data)
    cur.close()
    con.close()





flagofsearch=0  #两个flag
file_name="./城市数据.txt"
filename123="./logging.txt"
logging.basicConfig(filename=filename123,level=logging.INFO,format="%(asctime)s-%(levelname)s:%(message)s")
print("****************************初始化菜单****************************")
print("1:重新输入城市数据文件路径")
print("2:取消模糊匹配支持（输入城市名称后仅输出认为最大匹配度的城市）")
print("3:启用模糊匹配")
print("4：追加补充城市名称")
print("5：退出菜单")
print("*****************************************************************")
tmpp=tmpp=int(input("输入1-5选择所要执行的"))
while tmpp!=4 :
    if(tmpp==1):
        file_name=input("请输入新的文件路径")
        logging.info("重新选取了数据文件路径")
    elif(tmpp==2):
        flagofsearch=0
        logging.info("取消了模糊匹配")
        print("模糊匹配已取消")
    elif(tmpp==3):
        flagofsearch=0
        logging.info("开启了模糊匹配")
        print("模糊匹配已开启")
    elif(tmpp==4)  :
        cityname=input("请输入城市名称:")
        f1= open(file_name,'a',encoding='utf-8')
        f1.write(','+cityname)
        f1.close()
    elif(tmpp==5):
        logging.info("初始化结束！")
        print("初始化结束！")
        break
    else :
        print("输入错误！请重新输入！")
    tmpp=int(input("输入1-5选择所要执行的"))

f = open(file_name,encoding='utf-8')
data1=f.read()

t = 20
print("*******************************加载城市数据*******************************")
start = time.perf_counter()
for i in range(t + 1):
    finsh = "▓" * i
    need_do = "-" * (t - i)
    progress = (i / t) * 100
    dur = time.perf_counter() - start
    print("\r{:^3.0f}%[{}->{}]{:.2f}s".format(progress, finsh, need_do, dur), end="")
    time.sleep(0.05)
logging.info("进度条加载完毕！")

window=tk.Tk()
# style = Style(theme='pulse')
# window=style.master
city=''
cnt=0

window.title('省会二手房信息搜索')
window.geometry('1305x530')

window.tk.call("source", "azure.tcl")
window.tk.call("set_theme", "light")

window.maxsize(1320,535)
window.minsize(1305,530)
a=House(window)
window.mainloop()




****************************初始化菜单****************************
1:重新输入城市数据文件路径
2:取消模糊匹配支持（输入城市名称后仅输出认为最大匹配度的城市）
3:启用模糊匹配
4：追加补充城市名称
5：退出菜单
*****************************************************************
初始化结束！
*******************************加载城市数据*******************************
100%[▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓->]1.25sok
ok


# 预测模型


In [13]:
import pandas as pd
import re
import tkinter as tk
import numpy as np
from tkinter import ttk
import tkinter.messagebox
from numpy import nan as NaN
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor
from sklearn.svm import LinearSVR
import difflib
def fuzzy_matching(texts, value):  
    texts = texts.split(",")
    texts_score = {}
    for i in texts:
        score = difflib.SequenceMatcher(None, i, value).quick_ratio()
        texts_score[i] = score
    texts_score = sorted(texts_score.items(), key=lambda x: x[1], reverse=False)
    po=1
    match_value =[]
    tmp12=0
    b=[x[1] for x in texts_score]
    for i in range(1,6):
        print(i,end=':')
        print(texts_score[-i])
        tmp12=1  
        match_value.append(int(texts_score[-i][0][texts_score[-i][0].index('{')+1:texts_score[-i][0].index('}')]))
    return match_value

#函数部分
def predictprice():
    global typeofmodel
    newdata=[]
    try:
        square = int(text1.get())
        year = int(text2.get())
    except:
        tkinter.messagebox.showinfo('提示','请输入正确且完整数据后再进行预测')
        return

    if year>2022 or year<1990 :
        tkinter.messagebox.showinfo('提示','请输入正确的建造年份(1990-2021)')
        return 
    if l1.current()<2 or l2.current()<2 or l3.current()<2 or l4.current()<2 or l5.current()<2:
        tkinter.messagebox.showinfo('提示','请设置完房屋信息后再进行预测')
        return 
    print(var4.get())
    newdata.append(featuredict[var4.get()])
    newdata.append(square)
    newdata.append(decorationdict[var3.get()])
    newdata.append(floordict[var2.get()])
    newdata.append(year)
    newdata.append(typedict[var1.get()])
    newdata.append(addressdict[var5.get()])
    newdata=[newdata]

    predict2 = model2.predict(newdata)
    predict1 = model.predict(newdata)
    predict3 = model3.predict(newdata)
    predict4 = model4.predict(newdata)
    print(predict1,predict2,predict3,predict4)
    if typeofmodel == 1:  
        tkinter.messagebox.showinfo('预测值','预测房屋价格为'+str(predict1).strip('[]')+'万元')
    elif typeofmodel == 2:
        tkinter.messagebox.showinfo('预测值','预测房屋价格为'+str(predict2).strip('[]')+'万元')
    elif typeofmodel == 3:
        tkinter.messagebox.showinfo('预测值','预测房屋价格为'+str(predict3).strip('[]')+'万元') 
    elif typeofmodel == 4:
        tkinter.messagebox.showinfo('预测值','预测房屋价格为'+str(predict4).strip('[]')+'万元')   
def fun1():
    global typeofmodel
    typeofmodel=1
    checkbt.select()
    checkbt2.deselect()
    checkbt3.deselect()
    checkbt4.deselect()
def fun2():
    global typeofmodel
    typeofmodel=2
    checkbt2.select()
    checkbt.deselect()
    checkbt3.deselect()
    checkbt4.deselect()
def fun3():
    global typeofmodel
    typeofmodel=3
    checkbt3.select()
    checkbt.deselect()
    checkbt2.deselect()
    checkbt4.deselect()
def fun4():
    global typeofmodel
    typeofmodel=4
    checkbt4.select()
    checkbt2.deselect()
    checkbt3.deselect()
    checkbt.deselect()

def searchfuzzy():
    global text
    try:
        square = int(text1.get())
        year = int(text2.get())
    except:
        tkinter.messagebox.showinfo('提示','请输入正确且完整数据后再进行预测')
        return

    if year>2022 or year<1990 :
        tkinter.messagebox.showinfo('提示','请输入正确的建造年份(1990-2021)')
        return 
    if l1.current()<2 or l2.current()<2 or l3.current()<2 or l4.current()<2 or l5.current()<2:
        tkinter.messagebox.showinfo('提示','请设置完房屋信息后再进行预测')
        return 
    def delButton(tree):
        x=tree.get_children()
        for item in x:
            # print(item)
            tree.delete(item)
    delButton(tree)
    val = var4.get()+'|'+text2.get()+'平米'+'|'+var3.get()+'|'+var2.get()+'|'+text1.get()+'年建'+'|'+var1.get()+'|'+var5.get()
    result = fuzzy_matching(text,val)
    count = 1
    for i in result:
        print(house[i])
        Data=[count,house[i][0],house[i][1],house[i][2],house[i][3],house[i][4],house[i][5]]
        tree.insert('', count,text='', values=Data)    
        count+=1
#函数部分结束

#初始化必要数据部分
typedict={}
floordict={}
decorationdict={}
featuredict={}
typecount = 1
floorcount = 1
decorationcount = 1
featurecount = 1
text = ''   #匹配数据库
house = []

df = pd.read_excel("./housedata.xlsx",engine='openpyxl')
info = df[5]
address = df[2]
price = df[3]
info = [x for x in info]    #读取数据
#地址数据初始化
addressdict = {}
addresscount = 1
address = [x for x in address]
address = [x[x.index('-')+1:] for x in address]
for i in address:
    if i not in addressdict:
        addressdict[i] = addresscount
        addresscount+=1
# 相似度匹配的初始化数据
p=df[5]
p = [x.split('|') for x in p]
for i, items in enumerate(p): 
    del(p[i][2])
    p[i] = [item.strip() for item in items]  #清楚空格
    p[i]="|".join(p[i])

for i, items in enumerate(p): 
    p[i] = items+'|'+address[i] +'{'+ str(i) +'}'
  
for i in p:
    text = text+ i.replace(" ",'') + ','
ww1=df[0]
ww2=df[1]
ww3=df[2]
ww4=df[3]
ww5=df[4]
ww6=df[5]
for i in range(len(p)):
    house.append([ww1[i],ww2[i],ww3[i],ww4[i],ww5[i],ww6[i]])
# 初始化结束



price = [x for x in price]
info  = [x.split('|') for x in info]
for i, items in enumerate(info):               #处理数据为数值型 下列代码全是
    info[i] = [item.strip() for item in items]  #清楚空格
    if len(info[i])==6:
        info[i].insert(5,'2015年建')                    #补全数据
    if len(info[i])==8:
        del(info[i][-1])  
    if info[i][0] not in featuredict:    #几室几厅 处理特征
        featuredict[info[i][0]] = featurecount
        info[i][0]=featuredict[info[i][0]] 
        featurecount += 1 
    elif info[i][0] in featuredict:
        info[i][0] = featuredict[info[i][0]]

    info[i][1] = info[i][1].strip('平方米')         #1为房屋的面积
    info[i][5] = re.sub('[年建]','',info[i][5])
    
    if info[i][6] not in typedict:  #房屋类型没有出现
        typedict[info[i][6]] = typecount #添加新房屋类型进字典
        info[i][6]=typedict[info[i][6]]
        typecount += 1   #计数器加1
    elif info[i][6] in typedict:
        info[i][6] = typedict[info[i][6]]  #已有的直接转换数值型

    tmp=info[i][4][0]
    if tmp != '高' and tmp != '中' and tmp != '低':
        floor = int(info[i][4][:-1])
        if floor < 10:
            info[i][4]='低'
        elif floor <20:
            info[i][4]='中'
        else:
            info[i][4]='高'
    else:
        info[i][4] = info[i][4][0]
    if info[i][4] not in floordict: #楼层高低
        floordict[info[i][4]] = floorcount
        info[i][4]=floordict[info[i][4]]
        floorcount += 1 
    elif info[i][4] in floordict:
        info[i][4] = floordict[info[i][4]]

    if info[i][3] not in decorationdict: #装饰
        decorationdict[info[i][3]] = decorationcount
        info[i][3]=decorationdict[info[i][3]]
        decorationcount += 1         
    elif info[i][3] in decorationdict:
        info[i][3] = decorationdict[info[i][3]]
    del(info[i][2])    #房屋朝向删除 一个种类多且对房价影响较小的因素
    info[i].append(addressdict[address[i]])
    #数据预处理结束


X = info
y = price
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
model = LinearRegression()   #线性回归
model.fit(X_train, y_train)

model2 = LinearSVR()         #SVR回归
model2.fit(X_train, y_train)
    
model3 = DecisionTreeRegressor(max_depth=6,min_samples_leaf=2) #决策树回归
model3.fit(X_train, y_train)

model4 = MLPRegressor(hidden_layer_sizes=(10,), max_iter=5000) #神经网络模型
model4.fit(X_train, y_train)
#界面设计：
window = tk.Tk()
#选择属性
window.tk.call("source", "azure.tcl")
window.tk.call("set_theme", "light")
#房屋属性选择设置
var1 = tkinter.StringVar()
l1=ttk.Combobox(window,height=7,textvariable=var1)
l1['value'] = ('请选择楼房类型','------------------','板楼','板塔结合','塔楼')
l1.current(0)

var2 = tkinter.StringVar()
l2=ttk.Combobox(window,height=7,textvariable=var2)
l2['value'] = ('请选择楼层高低','------------------','高','中','低')
l2.current(0)

var3 = tkinter.StringVar()
l3=ttk.Combobox(window,height=7,textvariable=var3)
l3['value'] = ('请选择装修类型','------------------','其他','精装','毛坯','简装')
l3.current(0)

var4 = tkinter.StringVar()
l4=ttk.Combobox(window,height=7,textvariable=var4)
l4['value'] = ('请选择户型','------------------','1室0厅','1室1厅','1室2厅','2室1厅','2室2厅','3室1厅','3室2厅','3室3厅','4室1厅','4室2厅','4室3厅','4室4厅','5室2厅','5室3厅','6室1厅','6室2厅','6室3厅')
l4.current(0)

var5 = tkinter.StringVar()
l5 = ttk.Combobox(window,height=7,textvariable=var5)
l5['value'] = ('请选择区域','------------------','西三台子', '于洪广场', '奥体', '新市府', '长白', '棋盘山', '道义', '经济技术开发区', '会展中心', '四台子', '塔湾', '长青', '东湖', '于洪新城', '北陵', '白塔', '蒲河新城', '启工街', '三台子', '八王寺', '八一', '工人村', '北行', '兴工街', '东北大马路', '望花', '黎明', '北站北', '丁香湖', '泉园', '保工北', '铁西广场', '二十一世纪', '陵西', '张士', '艳粉', '营城子', '重工街', '东五里河', '正良', '市府', '北二路', '霁虹', '滑翔', '南市场', '新立堡', '三好街', '荷兰村', '新南站', '保工南', '云峰', '长客', '方家栏', '南湖', '陵东', '金廊', '北一路', '上园', '东中街', '二台子', '和平湾', '首府新区', '33号 -于洪广场', '新华广场', '南塔', '陶瓷城', '造化', '金融中心', '理工大学')
l5.current(0)

l1.grid(row=1,column=0,pady=15,padx=15)
l2.grid(row=1,column=1,pady=15,padx=15)
l3.grid(row=1,column=3,pady=15,padx=15)
l4.grid(row=1,column=4,pady=15,padx=15)
l5.grid(row=3,column=0,pady=15,padx=15)
#输入面积和日期控件以及文本控件
text1 = tk.Entry(window,show=None,width=30)
text2 = tk.Entry(window,show=None,width=30)
label1 = tk.Label(window,text='请输入房屋面积(平方米):',font='宋体')
label2 = tk.Label(window,text='请输入房屋的建造日期:',font='宋体')

label1.grid(row=2,column=0)
text1.grid(row=2,column=1)
label2.grid(row=2,column=3)
text2.grid(row=2,column=4)

label3 = tk.Label(window,text='沈阳二手房价格预测:',font='宋体',)
label3.grid(row=0,column=0)
#按钮预测房屋
bt1 = tk.Button(window,text='预测房价',width=10,height=1,command=predictprice,font='宋体',background='pink',foreground='blue')
bt1.grid(row=3,column=3)
#按钮匹配相似房屋
bt2 = tk.Button(window,text='匹配相似房屋',command=searchfuzzy,font='宋体',background='pink',foreground='red')
bt2.grid(row=3,column=4)
#两个不同模型的切换
label4 = tk.Label(window,text='选择模型:',font='宋体')
checkbt=tk.Checkbutton(window,text='SVR回归',command=fun1,font='宋体')
checkbt2=tk.Checkbutton(window,text='多元线性回归（最小二乘法）',command=fun2,font='宋体')
checkbt3=tk.Checkbutton(window,text='神经网络回归',command=fun3,font='宋体')
checkbt4=tk.Checkbutton(window,text='决策树回归',command=fun4,font='宋体')
checkbt.select()
typeofmodel=1  #模型有四种 linear，svr模型，神经网络模型，决策树模型
checkbt.grid(row=0,column=3,sticky='e')
checkbt2.grid(row=0,column=4)
checkbt3.grid(row=0,column=3,sticky='w')
checkbt4.grid(row=0,column=2,sticky='e')
label4.grid(row=0,column=1)
#树形控件对相似房屋进行展示
columns={
'ID':25,
'城市':40,
'广告标题':350,
'地址':200,
'总价/万元':50,
'每平方价格':100,
'主要信息':450
}
tree=ttk.Treeview(window,height=18,show='headings')
tree['columns'] = list(columns)
for column in columns:  # 批量设置列属性
    tree.heading(column, text=column)  # #设置列标题
    tree.column(column, width=columns[column], anchor='center')  
tree.grid(rows=5,columnspan=5,column=0)

window.mainloop()

c:\Users\guoxiaoke\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\utils\validation.py:976: FutureWarning: Arrays of bytes/strings is being converted to decimal numbers if dtype='numeric'. This behavior is deprecated in 0.24 and will be removed in 1.1 (renaming of 0.26). Please convert your data to numeric values explicitly instead.
  estimator=estimator,
c:\Users\guoxiaoke\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\svm\_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,


3室3厅
[253.0419283] [254.55506375] [129.33333333] [250.66164787]
1:('3室2厅|184.41平米|毛坯|中楼层(共30层)|2009年建|塔楼|奥体{356}', 0.7297297297297297)
2:('2室1厅|53.17平米|毛坯|高楼层(共33层)|2019年建|板塔结合|会展中心{1020}', 0.717948717948718)
3:('3室2厅|144平米|精装|中楼层(共33层)|2015年建|板塔结合|奥体{459}', 0.7123287671232876)
4:('1室1厅|58.91平米|精装|中楼层(共32层)|2014年建|板塔结合|奥体{1130}', 0.7105263157894737)
5:('2室1厅|77.76平米|毛坯|中楼层(共33层)|2017年建|板塔结合|新市府{900}', 0.7105263157894737)
['沈阳', '浑南 奥体板块 双地铁 清水房 南北通透 格局方正', 'SR国际新城 -奥体', 110.0, '5,965元/平', '3室2厅 | 184.41平米 | 南 北 | 毛坯 | 中楼层(共30层) | 2009年建 | 塔楼']
['沈阳', '奥园学府，刚需小两室，全明户型，视野好', '奥园国际城 -会展中心', 31.0, '5,831元/平', '2室1厅 | 53.17平米 | 南 西南 | 毛坯 | 高楼层(共33层) | 2019年建 | 板塔结合']
['沈阳', '浑南奥体 华发 看房方便 房子真的不错', '华发首府 -奥体', 179.0, '12,431元/平', '3室2厅 | 144平米 | 南 北 | 精装 | 中楼层(共33层) | 2015年建 | 板塔结合']
['沈阳', '奥体商圈 独立一室一厅  着急出售 满五年', '名流印象四五期 -奥体', 59.0, '10,016元/平', '1室1厅 | 58.91平米 | 东 | 精装 | 中楼层(共32层) | 2014年建 | 板塔结合']
['沈阳', '南北通透标准好格局，采光充足，视野开阔', '首创光和城四期 -新市府', 74.0, '9,517元/平', '2室1厅 | 77.76平米 | 南 北 | 毛坯 

In [15]:
from sklearn.model_selection import GridSearchCV

# 定义要调参的参数值
param_grid = {'max_depth': [2, 4, 6, 8, 10], 'min_samples_leaf': [2, 4, 6, 8, 10]}

# 创建决策树模型
model = DecisionTreeRegressor()

# 使用GridSearchCV进行调参
grid_search = GridSearchCV(model, param_grid, cv=5)

# 在训练集上训练
grid_search.fit(X_train, y_train)

# 输出最优的超参数组合
print(grid_search.best_params_)

{'max_depth': 10, 'min_samples_leaf': 4}
